# Data for Good: predicting suicidal behavior likelihood among Reddit users using Deep Learning (Part 2)

*Deep Learning and Reinforcement Learning (part of IBM Machine Learning Professional Certificate) - Course Project.*

>*No one is useless in this world who lightens the burdens of another.*  
― **Charles Dickens**

<img src='https://www.discover-norway.no/upload/images/-development/header/desktop/kul_munch/edvard%20munch%20the%20scream%201893_munchmmuseet.jpg'></img>

## Table of contents
1. [Data Preparation](#preparation)  
2. [Model Development: Recurrent Neural Network](#model)  
  2.1. [...](#kmeans)  
  2.2. [...](#hac)  
  2.3. [...](#dbscan)  
3. [Results](#results)  
4. [Discussion](#discussion)  
5. [Conclusion](#conclusion)  
  5.1. [Project Summary](#summary)  
  5.2. [Outcome of the Analysis](#outcome)  
  5.3. [Potential Developments](#developments)

## 1. Data Preparation <a name=preparation></a>

Steps to process the data for modeling:
1. Drop not-relevant dataset features.
2. Remove from data the stopwords found during the word cloud analysis.
3. Tokenize the posts.
4. One-Hot Encode the target variable (the classes)
5. Pad the sequences.
6. Split dataset into training and testing sets.

In [1]:
#Import needed libraries
import keras
import pandas as pd
import random
from random import randrange, seed
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
#Import data (after cleaning and the EDA perfomed in word-cloud environment notebook)
data = pd.read_csv(r'data.csv')
processed_data = data.copy()
processed_data.head()

,User,Post,Label,word_count
0,user-0,its not a viable option and youll be leaving y...,Supportive,134
1,user-1,it can be hard to appreciate the notion that y...,Ideation,2163
2,user-2,hi so last night i was sitting on the ledge of...,Behavior,470
3,user-3,i tried to kill my self once and failed badly ...,Attempt,885
4,user-4,hi nem3030 what sorts of things do you enjoy d...,Ideation,208


##### 1. Drop not-relevant features.

In [3]:
#Drop not relevant features
processed_data.drop(['User', 'word_count'], axis=1, inplace=True)
processed_data.tail()

,Post,Label
495,its not the end it just feels that way or at l...,Supportive
496,it was a skype call but she ended it and ventr...,Indicator
497,that sounds really weird maybe you were distra...,Supportive
498,dont know there as dumb as it sounds i feel hy...,Attempt
499,gt it gets better trust me ive spent long enou...,Behavior


###### 2. Remove the stop words.

I start processing the data by deleting the stopwords found during the word cloud analysis (see Part 1 Notebook).

In [4]:
#Import the stop_words list and create a Python list
stop_words = open(r'stop_words.txt', 'r')
sw=[]
for line in stop_words:
    sw.append(line[:-1])
    
print('Length of stop word list:', len(sw))

Length of stop word list: 323


In [5]:
#Close the file
stop_words.close()
print('Is the file closed?', stop_words.closed)

Is the file closed? True


In [6]:
print("First 50 stop words:\n",sw[:51])

First 50 stop words:
 ['see', "where's", 'not', 'm', 'have', 'whom', 'need', 'maybe', 'to', 'someone', 'get', 'which', "aren't", 'our', 'made', 'like', "weren't", 'hasn', 'won', "you'd", "isn't", 'nor', 'back', "here's", 'my', 'else', 'too', 'shouldn', 'always', "who's", 'day', "we'd", 'how', "how's", 'years', 'since', 'happy', 'was', 'friends', 'under', "i've", 'www', 'try', 'thats', "you'll", 'give', 'yours', "we've", 'ever', 'll', 'd']


In [7]:
#let's visualize a random post
random.seed(3)
processed_data.loc[randrange(500)]['Post']

'no more ideas i dont agree with live for others kind of advice i think you should live for yourself and your friends and family the world isnt going to be fixed any time soon so stop thinking its all on your shoulders regular exercise and a lack of excessive stress is important to a good life so is a decent job work is now stressful yes its never done im on a long break now its tired hot and humid where i now live so i cant really do anything i cant handle the heat well i want to prepare for my death before i go back to work its not only that the career enabled me to live a certain lifestyle and live in a certain place and not have to worry too much about money and other things why would you like that i dont think there are any other kinds of job i could do in this country it has been 5 years since i lost my job i have tried my best the things i lost in my life i believe them to be extremely fundamental and important things i also lost a life that had little worry and stress now i hav

In [8]:
random.seed(3)
print('Length of the post before removing the stop words:', len(processed_data.loc[randrange(500)]['Post']))

Length of the post before removing the stop words: 2269


In [9]:
#let's remove the stop words
processed_data['Post'] = processed_data['Post'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

#let's visualize the same post without stopwords
random.seed(3)
processed_data.loc[randrange(500)]['Post']

'ideas agree others kind advice family world fixed soon stop thinking shoulders regular exercise lack excessive stress important decent job stressful yes done break tired hot humid handle heat prepare death career enabled certain lifestyle certain place worry money kinds job country 5 lost job tried best lost believe extremely fundamental important lost little worry stress job gets worse allow exercise boiling hot city saps energy horrible bitchy colleagues norm realize liked living country kind jobs worse world shitty jobs best jobs world threw tolerate job rest move different job industry city less hot humid place wont climate city ill lost suicide arent suicide attempt looked upon mental asthenia moment madness kind childish gesture arent actual suicides imagine kill guess lack understanding survival mechanism suicidal likely fixed world fucked 7bn fucking planet mere presence forget enjoy'

In [10]:
random.seed(3)
print('Length of the post after removing the stop words:', len(processed_data.loc[randrange(500)]['Post']))

Length of the post after removing the stop words: 904


###### 3. Tokenize the text.

I am going to tokenize the posts, that is I'll turn the text into a list of individual words and then convert the words into integers, using the Keras Tokenizer class.

In [11]:
#let's visualize a random post
random.seed(13)
processed_data.loc[randrange(500)]['Post']

'dude wont called brave bold become guy killed body buck news die ill kick ass heaven whever'

In [12]:
#Let's tokenize the data
tokenizer = Tokenizer()
#train the tokenizer
tokenizer.fit_on_texts(processed_data['Post'])
#conver text into lists of integers
posts = tokenizer.texts_to_sequences(processed_data['Post'])

In [13]:
#let's visualize the same post after tokenizing
random.seed(13)
print(posts[randrange(500)])

[579, 39, 342, 1008, 6259, 178, 143, 725, 326, 6260, 1059, 75, 17, 1188, 770, 2454, 11488]


In [14]:
#Let's map the intetgers back to words to check integer meaning
random.seed(13)
' '.join(tokenizer.index_word[w] for w in posts[randrange(500)])

'dude wont called brave bold become guy killed body buck news die ill kick ass heaven whever'

###### 4. One-Hot Encode the target variable.

I now one-hot encode, using Keras library, the data classes

In [15]:
processed_data['class'] =  processed_data['Label'].apply(lambda x: 1 if x == 'Supportive' else 2 if x == 'Indicator'
                                                         else 3 if x == 'Ideation' else 4 if x == 'Behavior' else 5 )

output = keras.utils.to_categorical(processed_data['class'])
output = output[:,1:]
output

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]], dtype=float32)

###### 5. Pad the sequences.

Let's now create sequences of the same lenght. During the Exploratory Data Analysis we have foud out the 80% of posts have fewer than 2,000 words. Therefore I set the maximum sequence length as 2,000: post longer than 2,000 words will be truncated, whilst posts shorter then 2,000 words will be padded.

In [16]:
posts = pad_sequences(posts, maxlen=2000, padding='post', truncating='post')
#posts[0]

###### 6. Split the dataset into training and testing sets.

Let's now create the final dataset ready for modelling, by concatenating the tokenized word sequences with the encoded classes:

In [17]:
model_data = np.concatenate((posts, output), axis=1)
np.shape(model_data)

(500, 2005)

Let's count now the total number of words that our dataset contains. This is the size of our entire vocabulary.

In [18]:
num_words = len(np.unique(posts))
print('After the pre-processing stage, the data contains {} unique words'.format(f'{num_words:,}'))

After the pre-processing stage, the data contains 17,452 unique words


Let's split the dataset into train and test sets. I use 20% of the dataset (100 observations) as test data, and the stratify parameter to preserve the class imbalance.

In [19]:
x_train, x_test, y_train, y_test = train_test_split(model_data[:,:-5], model_data[:,-5:], test_size=0.2, random_state=666,
                                                    stratify = model_data[:,-5:])

In [20]:
print('Training dataset shape:', x_train.shape)
print('Testing dataset shape:', x_test.shape)

Training dataset shape: (400, 2000)
Testing dataset shape: (100, 2000)


## 2. Model Development <a name= 'model'></a>

Model hyperparameters:
- embeddeding layer dimensions and train/pretrained
- number of layers before/after the recorrent section of the network
- the state dimension
- RNN initializersL default
- number of neurons in the hidden layer(s)
- activation functions for the hidden layers (sigmoid, tangent, relu, leaky relu)
- learning rate
- bach size (usually 16 or 32)
- number of epochs
- regularization: stochastic or mini-batch (evaluate other regularization techinque only if the model overfits the data)
- optimizers

In [21]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from numpy.random import seed
import tensorflow
seed(1)
tensorflow.random.set_seed(1)

In [22]:
#Initialize the model
plain_rnn = Sequential()

# Add the Embedding layer, which maps each input integer (word) to a 50-dimensional vector.
#I am not using any pre-trained embeddings
plain_rnn.add(Embedding(posts.max()+1, output_dim=300, trainable=True, mask_zero=True))

# Add the RNN layer
plain_rnn.add(SimpleRNN(units=150, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', activation='tanh',
                        input_shape=x_train.shape[1:]))

# Add the more dense layers and the final output layer
plain_rnn.add(Dense(5, activation='softmax'))

# Compile the model
adam = keras.optimizers.Adam(learning_rate=0.001)
plain_rnn.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

#Let's check the model architecture
plain_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         5248800   
                                                                 
 simple_rnn (SimpleRNN)      (None, 150)               67650     
                                                                 
 dense (Dense)               (None, 5)                 755       
                                                                 
Total params: 5,317,205
Trainable params: 5,317,205
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Train the model and seed the model to get reprducible results
plain_rnn.fit(x_train, y_train, batch_size=16, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/10
25/25 [==============================] - 38s 1s/step - loss: 1.5772 - accuracy: 0.2750 - val_loss: 1.5537 - val_accuracy: 0.3300
Epoch 2/10
25/25 [==============================] - 31s 1s/step - loss: 0.8170 - accuracy: 0.8625 - val_loss: 1.6042 - val_accuracy: 0.3200
Epoch 3/10
25/25 [==============================] - 32s 1s/step - loss: 0.4503 - accuracy: 0.9525 - val_loss: 1.8274 - val_accuracy: 0.1900
Epoch 4/10
25/25 [==============================] - 31s 1s/step - loss: 0.1922 - accuracy: 0.9950 - val_loss: 1.7003 - val_accuracy: 0.2700
Epoch 5/10
25/25 [==============================] - 33s 1s/step - loss: 0.0495 - accuracy: 1.0000 - val_loss: 1.7960 - val_accuracy: 0.2800
Epoch 6/10
25/25 [==============================] - 31s 1s/step - loss: 0.0260 - accuracy: 1.0000 - val_loss: 1.8554 - val_accuracy: 0.2800
Epoch 7/10
25/25 [==============================] - 32s 1s/step - loss: 0.0188 - accuracy: 1.0000 - val_loss: 1.8880 - val_accuracy: 0.2700
Epoch 8/10
25/25 [==

In [24]:
#To output the validation set loss and metrics
plain_rnn.evaluate(x_test, y_test)

4/4 [==============================] - 2s 365ms/step - loss: 1.9683 - accuracy: 0.2300


[1.9683005809783936, 0.23000000417232513]

---

Model 2: reducing complexity by reducing parameters, and using dropout.

In [154]:
posts_100 = pad_sequences(posts, maxlen=100, padding='post', truncating='post')

In [155]:
model_data = np.concatenate((posts_100, output), axis=1)
np.shape(model_data)

(500, 105)

Let's count now the total number of words that our dataset contains. This is the size of our entire vocabulary.

In [156]:
num_words = len(np.unique(posts_100))
print('After the pre-processing stage, the data contains {} unique words'.format(f'{num_words:,}'))

After the pre-processing stage, the data contains 7,656 unique words


Let's split the dataset into train and test sets. I use 20% of the dataset (100 observations) as test data, and the stratify parameter to preserve the class imbalance.

In [157]:
x_train, x_test, y_train, y_test = train_test_split(model_data[:,:-5], model_data[:,-5:], test_size=0.2, random_state=50,
                                                    stratify = model_data[:,-5:])

In [158]:
print('Training feature dataset shape:', x_train.shape)
print('Testing feature dataset shape:', x_test.shape)

Training feature dataset shape: (400, 100)
Testing feature dataset shape: (100, 100)


In [159]:
print('Training class dataset shape:', y_train.shape)
print('Testing class dataset shape:', y_test.shape)

Training class dataset shape: (400, 5)
Testing class dataset shape: (100, 5)


In [160]:
seed(2)
tensorflow.random.set_seed(2)

#Initialize the model
rnn_2 = Sequential()

# Add the Embedding layer, which maps each input integer (word) to a 50-dimensional vector.
#I am not using any pre-trained embeddings
rnn_2.add(Embedding(posts_100.max()+1, output_dim=50, trainable=True, mask_zero=True))

# Add the RNN layer
rnn_2.add(SimpleRNN(units=10, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', activation='tanh',
                        input_shape=x_train.shape[1:], dropout=.2, recurrent_dropout=.2))

# Add the more dense layers and the final output layer
rnn_2.add(Dense(5, activation='softmax'))

# Compile the model
adam = keras.optimizers.Adam(learning_rate=0.001)
rnn_2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

#Let's check the model architecture
rnn_2.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, None, 50)          874800    
                                                                 
 simple_rnn_21 (SimpleRNN)   (None, 10)                610       
                                                                 
 dense_21 (Dense)            (None, 5)                 55        
                                                                 
Total params: 875,465
Trainable params: 875,465
Non-trainable params: 0
_________________________________________________________________


In [161]:
rnn_2.fit(x_train, y_train, batch_size=16, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/10
25/25 [==============================] - 6s 63ms/step - loss: 1.6586 - accuracy: 0.2025 - val_loss: 1.6238 - val_accuracy: 0.1800
Epoch 2/10
25/25 [==============================] - 1s 42ms/step - loss: 1.6174 - accuracy: 0.2275 - val_loss: 1.6254 - val_accuracy: 0.2500
Epoch 3/10
25/25 [==============================] - 1s 40ms/step - loss: 1.5457 - accuracy: 0.3125 - val_loss: 1.6279 - val_accuracy: 0.1800
Epoch 4/10
25/25 [==============================] - 1s 40ms/step - loss: 1.5223 - accuracy: 0.3650 - val_loss: 1.6296 - val_accuracy: 0.1600
Epoch 5/10
25/25 [==============================] - 1s 42ms/step - loss: 1.4446 - accuracy: 0.4175 - val_loss: 1.6230 - val_accuracy: 0.1800
Epoch 6/10
25/25 [==============================] - 1s 44ms/step - loss: 1.4149 - accuracy: 0.4425 - val_loss: 1.6201 - val_accuracy: 0.2000
Epoch 7/10
25/25 [==============================] - 1s 44ms/step - loss: 1.3747 - accuracy: 0.4850 - val_loss: 1.6082 - val_accuracy: 0.2100
Epoch 8/10
25

In [162]:
rnn_2.fit(x_train, y_train, batch_size=16, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/10
25/25 [==============================] - 1s 44ms/step - loss: 1.1503 - accuracy: 0.6950 - val_loss: 1.6034 - val_accuracy: 0.2300
Epoch 2/10
25/25 [==============================] - 1s 41ms/step - loss: 1.0662 - accuracy: 0.7500 - val_loss: 1.6055 - val_accuracy: 0.2500
Epoch 3/10
25/25 [==============================] - 1s 41ms/step - loss: 1.0190 - accuracy: 0.7675 - val_loss: 1.6074 - val_accuracy: 0.2400
Epoch 4/10
25/25 [==============================] - 1s 41ms/step - loss: 0.9373 - accuracy: 0.8225 - val_loss: 1.6081 - val_accuracy: 0.2400
Epoch 5/10
25/25 [==============================] - 1s 44ms/step - loss: 0.8969 - accuracy: 0.8225 - val_loss: 1.6117 - val_accuracy: 0.2400
Epoch 6/10
25/25 [==============================] - 1s 45ms/step - loss: 0.8281 - accuracy: 0.8750 - val_loss: 1.6216 - val_accuracy: 0.2500
Epoch 7/10
25/25 [==============================] - 1s 41ms/step - loss: 0.7743 - accuracy: 0.8825 - val_loss: 1.6361 - val_accuracy: 0.2300
Epoch 8/10
25

In [163]:
rnn_2.fit(x_train, y_train, batch_size=16, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/10
25/25 [==============================] - 1s 49ms/step - loss: 0.6010 - accuracy: 0.8900 - val_loss: 1.6713 - val_accuracy: 0.2500
Epoch 2/10
25/25 [==============================] - 1s 45ms/step - loss: 0.5312 - accuracy: 0.9525 - val_loss: 1.6814 - val_accuracy: 0.2300
Epoch 3/10
25/25 [==============================] - 1s 48ms/step - loss: 0.5050 - accuracy: 0.9425 - val_loss: 1.6982 - val_accuracy: 0.2300
Epoch 4/10
25/25 [==============================] - 1s 42ms/step - loss: 0.4509 - accuracy: 0.9450 - val_loss: 1.7170 - val_accuracy: 0.2200
Epoch 5/10
25/25 [==============================] - 1s 41ms/step - loss: 0.4208 - accuracy: 0.9575 - val_loss: 1.7305 - val_accuracy: 0.2300
Epoch 6/10
25/25 [==============================] - 1s 42ms/step - loss: 0.4025 - accuracy: 0.9600 - val_loss: 1.7480 - val_accuracy: 0.2200
Epoch 7/10
25/25 [==============================] - 1s 44ms/step - loss: 0.3524 - accuracy: 0.9675 - val_loss: 1.7601 - val_accuracy: 0.2100
Epoch 8/10
25

In [164]:
rnn_2.fit(x_train, y_train, batch_size=16, epochs=5, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/5
25/25 [==============================] - 1s 52ms/step - loss: 0.2471 - accuracy: 0.9875 - val_loss: 1.8137 - val_accuracy: 0.2300
Epoch 2/5
25/25 [==============================] - 1s 45ms/step - loss: 0.2566 - accuracy: 0.9825 - val_loss: 1.8296 - val_accuracy: 0.2200
Epoch 3/5
25/25 [==============================] - 1s 44ms/step - loss: 0.2251 - accuracy: 0.9850 - val_loss: 1.8457 - val_accuracy: 0.2200
Epoch 4/5
25/25 [==============================] - 1s 48ms/step - loss: 0.2247 - accuracy: 0.9850 - val_loss: 1.8615 - val_accuracy: 0.2100
Epoch 5/5
25/25 [==============================] - 1s 48ms/step - loss: 0.2058 - accuracy: 0.9975 - val_loss: 1.8794 - val_accuracy: 0.2100


In [165]:
rnn_2.fit(x_train, y_train, batch_size=16, epochs=5, shuffle=True, validation_data=(x_test, y_test))

Epoch 1/5
25/25 [==============================] - 1s 46ms/step - loss: 0.1838 - accuracy: 0.9925 - val_loss: 1.8954 - val_accuracy: 0.2100
Epoch 2/5
25/25 [==============================] - 1s 49ms/step - loss: 0.1815 - accuracy: 0.9925 - val_loss: 1.9136 - val_accuracy: 0.2100
Epoch 3/5
25/25 [==============================] - 1s 44ms/step - loss: 0.1504 - accuracy: 0.9925 - val_loss: 1.9332 - val_accuracy: 0.2000
Epoch 4/5
25/25 [==============================] - 1s 48ms/step - loss: 0.1431 - accuracy: 0.9950 - val_loss: 1.9485 - val_accuracy: 0.2100
Epoch 5/5
25/25 [==============================] - 1s 45ms/step - loss: 0.1462 - accuracy: 0.9950 - val_loss: 1.9609 - val_accuracy: 0.2100


---

In [172]:
y_pred_train = rnn_2.predict(x_train)
sum(np.argmax(y_train, axis=1) == np.argmax(y_pred_train, axis=1))

13/13 [==============================] - 0s 9ms/step


400

In [173]:
y_pred = rnn_2.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred

4/4 [==============================] - 0s 12ms/step


array([2, 1, 4, 2, 3, 3, 2, 1, 4, 3, 4, 2, 2, 2, 2, 0, 0, 0, 3, 3, 0, 2,
       2, 3, 2, 3, 2, 3, 2, 2, 1, 3, 1, 1, 3, 2, 2, 0, 0, 4, 0, 0, 3, 1,
       3, 1, 4, 0, 2, 2, 1, 2, 2, 1, 3, 1, 1, 0, 2, 2, 0, 1, 2, 0, 0, 2,
       2, 3, 0, 3, 3, 3, 2, 3, 4, 1, 0, 3, 1, 1, 0, 1, 0, 3, 1, 2, 3, 2,
       0, 3, 1, 1, 0, 2, 3, 1, 2, 0, 0, 1], dtype=int64)

In [246]:
np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 0)[0]],return_counts=True)[1]

array([5, 4, 8, 3, 2], dtype=int64)

In [250]:
2/22

0.09090909090909091

In [245]:
#actual class: supportive (0)
i=0
for v in np.unique(y_pred[np.where(np.argmax(y_test, axis=1) == 0)]):
    print(v,
          np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 0)[0]],return_counts=True)[1][i]/
          len(np.where(np.argmax(y_test,axis=1)==0)[0]))
    i+=1

0 0.22727272727272727
1 0.18181818181818182
2 0.36363636363636365
3 0.13636363636363635
4 0.09090909090909091


In [252]:
np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 1)[0]],return_counts=True)

(array([0, 1, 2, 3], dtype=int64), array([8, 3, 6, 3], dtype=int64))

In [255]:
len(np.where(np.argmax(y_test,axis=1)==1)[0])

20

In [261]:
6/20

0.3

In [258]:
#actual class: indicator (1)
i=0
for v in np.unique(y_pred[np.where(np.argmax(y_test, axis=1) == 1)]):
    print(v,
          np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 1)[0]],return_counts=True)[1][i]/
          len(np.where(np.argmax(y_test,axis=1)==1)[0]))
    i+=1

0 0.4
1 0.15
2 0.3
3 0.15


In [256]:
np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 2)[0]],return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64), array([ 4,  6, 10, 11,  3], dtype=int64))

In [257]:
len(np.where(np.argmax(y_test,axis=1)==2)[0])

34

In [267]:
11/34

0.3235294117647059

In [262]:
#actual class: ideation (2)
i=0
for v in np.unique(y_pred[np.where(np.argmax(y_test, axis=1) == 2)]):
    print(v,
          np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 2)[0]],return_counts=True)[1][i]/
          len(np.where(np.argmax(y_test,axis=1)==2)[0]))
    i+=1

0 0.11764705882352941
1 0.17647058823529413
2 0.29411764705882354
3 0.3235294117647059
4 0.08823529411764706


In [263]:
np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 3)[0]],return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64), array([3, 5, 3, 3, 1], dtype=int64))

In [264]:
len(np.where(np.argmax(y_test,axis=1)==3)[0])

15

In [269]:
3/15

0.2

In [265]:
#actual class: behavior (3)
i=0
for v in np.unique(y_pred[np.where(np.argmax(y_test, axis=1) == 3)]):
    print(v,
          np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 3)[0]],return_counts=True)[1][i]/
          len(np.where(np.argmax(y_test,axis=1)==3)[0]))
    i+=1

0 0.2
1 0.3333333333333333
2 0.2
3 0.2
4 0.06666666666666667


In [270]:
np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 4)[0]],return_counts=True)

(array([0, 1, 2, 3], dtype=int64), array([1, 3, 2, 3], dtype=int64))

In [272]:
len(np.where(np.argmax(y_test,axis=1)==4)[0])

9

In [269]:
3/9

0.2

In [273]:
#actual class: attempt (4)
i=0
for v in np.unique(y_pred[np.where(np.argmax(y_test, axis=1) == 4)]):
    print(v,
          np.unique(y_pred[np.where(np.argmax(y_test,axis=1) == 4)[0]],return_counts=True)[1][i]/
          len(np.where(np.argmax(y_test,axis=1)==4)[0]))
    i+=1

0 0.1111111111111111
1 0.3333333333333333
2 0.2222222222222222
3 0.3333333333333333


Extras:
1. can I do cross validation / hyperparameters tuning with deep learnig models: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/#:~:text=By%20setting%20the%20n_jobs%20argument,for%20each%20combination%20of%20parameters.

---

sources for data-preprocessing (NLP):
- https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470
- https://medium0.com/@saad.arshad102/sentiment-analysis-text-classification-using-rnn-bi-lstm-recurrent-neural-network-81086dda8472

---

data source: https://www.kaggle.com/datasets/thedevastator/c-ssrs-labeled-suicidality-in-500-anonymized-red
https://zenodo.org/record/2667859#.Y9aqCXZBw2z